In [1]:
import sys
sys.path.append('../..')
from config.configs import ROOT_DIR

file_path = ROOT_DIR / 'data' / 'mark' / 'Fw__Examples_of_QC_reports_for_setting_up_templates_in_Leadtools' / 'DPS9 PO 177768 - 221234 Summary Report.pdf'
file_path

WindowsPath('d:/github/OCR/notebooks/openai/../../data/mark/Fw__Examples_of_QC_reports_for_setting_up_templates_in_Leadtools/DPS9 PO 177768 - 221234 Summary Report.pdf')

In [2]:
from openai import OpenAI

# Create a client
client = OpenAI()

vector_store = client.beta.vector_stores.create(name="Report Summaries")

file_streams = [open(file_path, "rb")]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_streams,
)

In [3]:
assistant = client.beta.assistants.create(
    name="PDF reporter",
    description="You are adept at summarizing PDF reports. You analyze data present in .pdf files, understand trends, and provide concise summaries.",
    model="gpt-4o",
    tools=[{"type": "file_search"}],
        tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

In [4]:
thread = client.beta.threads.create()

text = "Extract the all the items important content from the report table (Probably present in first 1 or 2 pages) in the PDF file and return as a dataframe."
messages = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=text,
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

In [5]:
messages = client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id)
message_content = messages.data[0].content[0].text
print(message_content.value)

Based on the content review of the specified pages in the PDF, here is the important item information extracted in the form of a dataframe:

| Column Name | Data |
|-------------|------|
| Customer    | DPS9 PO 177768 / ISS PO 221234 |
| Received Date | 20/01/2025 08:12:44 |
| Supplier    | INTERTERRA agromedina |
| Country of Origin | Spain |
| Pallet ID   | Supplier: 6187903, Customer: 6187903 |
| Product     | Persimon 15x3 Cell |
| Harvest Date | 09/11/2024 |
| Expected Quantity | 145 |
| Received Quantity | 145 |
| Estimated Yield | 100% |
| Defects Total | 3.70% |
| Minor Defects | Poor Colour: 2.22%, Scarring: 1.48% |
| Inspection | Conducted by: Bobby.Murphy |
| Lot Number | 551839 |
| Brand      | STND |
| End Customer | Marks & Spencer |
| Box/Pack Weights | Avg 371g, Min 355g, Max 403g |
| Sugar Brix | Avg 17.36%, Min 15.80%, Max 18.90% |
| Pressure   | Avg 3.78, Min 1.95, Max 5.32 |
| Comments   | Scarring and poor colour noted. |

This generalization follows the results se

In [6]:
messages

SyncCursorPage[Message](data=[Message(id='msg_lC8QMLbo61BZN5Wp2LaW36Ut', assistant_id='asst_KSclhP7d7JPL9YLzVXzilPz1', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=1108, file_citation=FileCitation(file_id='file-Ks81euNg4UJZJTFE6hVfJr'), start_index=1060, text='【4:0†DPS9 PO 177768 - 221234 Summary Report.pdf】', type='file_citation')], value='Based on the content review of the specified pages in the PDF, here is the important item information extracted in the form of a dataframe:\n\n| Column Name | Data |\n|-------------|------|\n| Customer    | DPS9 PO 177768 / ISS PO 221234 |\n| Received Date | 20/01/2025 08:12:44 |\n| Supplier    | INTERTERRA agromedina |\n| Country of Origin | Spain |\n| Pallet ID   | Supplier: 6187903, Customer: 6187903 |\n| Product     | Persimon 15x3 Cell |\n| Harvest Date | 09/11/2024 |\n| Expected Quantity | 145 |\n| Received Quantity | 145 |\n| Estimated Yield | 100% |\n| Defects Total | 3.